In [ ]:
!pip install mxnet
!pip install ignite
!pip install pytorch-ignite
!pip install metrics

In [1]:
echo "# Reproducibility" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/shamma315/Reproducibility.git
git push -u origin main

SyntaxError: ignored

In [ ]:
import os

#import metrics as metrics
import numpy as np

os.environ['PYOPENCL_CTX'] = '0'
from model import StructuredSelfAttention
from train import train
import torch
import utils as utils
from data_got import load_data
from tqdm import tqdm
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy
from ignite.contrib.handlers import ProgressBar
from sklearn.metrics import precision_score
from train import Ndcg_k
from train import precision_k

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
config = utils.read_config("config.yml")

device = torch.device('cuda')
# x = torch.randn(2, 3, device=device)
# x = x.to('cpu')

print("Using OPENCL")

print('loading data...\n')
label_num = 54
train_loader, test_loader, label_embed, embed, X_tst, word_to_id, Y_tst, Y_trn = load_data(batch_size=config.batch_size)
label_embed = torch.from_numpy(label_embed).float().to(device)  # [L*256]
embed = torch.from_numpy(embed).float().to(device)
print("load done")

attention_model = StructuredSelfAttention(batch_size=config.batch_size,lstm_hid_dim=config['lstm_hidden_dimension'],
                                          d_a=config["d_a"], n_classes=label_num, label_embed=label_embed,embeddings=embed)

def process_function(engine, batch):
    attention_model.train()
    x, y = batch
    x = x.to(device)
    y_true = y.to(device)
    y_pred = attention_model(x)
    y_pred = y_pred.float()
    y_true = y_true.float()
    # y = y.float()
    loss = loss_fn(y_pred, y_true)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

   #  # Compute precision and ndcg for the current batch
   #  y_pred_np = y_pred.detach().cpu().numpy()
   # # y_true = y.cpu().numpy()
   #  ndcg = Ndcg_k(y_pred.detach().cpu().numpy(), y_true, k=1)
   #  precision = precision_k(y_pred_np, y_true.cpu().numpy(), k=1)
   #  return {
   #      "loss": loss.item(),
   #      "ndcg": ndcg,
   #
   #      "precision": prec,
   #   }

def eval_function(engine, batch):
    attention_model.eval()
    with torch.no_grad():
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = attention_model(x)
        return y_pred, y

def output_transform(output):
    y_pred, y = output
    y_pred = torch.round(torch.sigmoid(y_pred))
    return y_pred, y

trainer = Engine(process_function)
evaluator = Engine(eval_function)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(attention_model.parameters(), lr=0.001, betas=(0.9, 0.99))
metric = Accuracy(output_transform=output_transform)

metric.attach(evaluator, 'accuracy')

pbar = ProgressBar(persist=True)
pbar.attach(trainer, ['loss'])

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    avg_loss = metric['loss']
    prec = np.mean(metrics['precision'], axis=0)
    ndcg = np.mean(metrics['ndcg'], axis=0)
    print("epoch %2d train end : avg_loss = %.4f, precision@1 = %.4f, precision@3 = %.4f, precision@5 = %.4f, ndcg@1 = %.4f, ndcg@3 = %.4f, ndcg@5 = %.4f" % (trainer.state.epoch, avg_loss, prec[0], prec[2], prec[4], ndcg[0], ndcg[2], ndcg[4]))
   ## accuracy = metrics['accuracy']
   ## print(f"Validation Results - Epoch: {trainer.state.epoch}  Avg accuracy: {accuracy:.2f}")

#if device.type == 'cuda':
#    attention_model = attention_model.to(device)
 #   loss_fn = loss_fn.to(device)

trainer.run(train_loader, max_epochs=config["epochs"])

Using OPENCL
loading data...



ERROR:ignite.engine.engine.Engine:Current run is terminating due to exception: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu
ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu


load done


RuntimeError: ignored

In [ ]:
if config.GPU:
    torch.cuda.set_device(0)
print('loading data...\n')
label_num = 54
train_loader, test_loader, label_embed,embed,X_tst,word_to_id,Y_tst,Y_trn = load_data(batch_size=config.batch_size)
label_embed = torch.from_numpy(label_embed).float()  # [L*256]
embed = torch.from_numpy(embed).float()
print("load done")

def multilabel_classification(attention_model, train_loader, test_loader, epochs, GPU=True):
    loss = torch.nn.BCELoss()
    opt = torch.optim.Adam(attention_model.parameters(), lr=0.001, betas=(0.9, 0.99))
    train(attention_model, train_loader, test_loader, loss, opt, epochs,GPU)

attention_model = StructuredSelfAttention(batch_size=config.batch_size, lstm_hid_dim=config['lstm_hidden_dimension'],
                                          d_a=config["d_a"], n_classes=label_num, label_embed=label_embed,embeddings=embed)
if config.use_cuda:
    attention_model.cuda()
multilabel_classification(attention_model, train_loader, test_loader, epochs=config["epochs"]) 

loading data...

load done
Running EPOCH 1


100%|██████████| 856/856 [02:21<00:00,  6.04it/s]


epoch  1 train end : avg_loss = 0.0015
precision@1 : 0.7108 , precision@3 : 0.5107 , precision@5 : 0.3576 
ndcg@1 : 0.7108 , ndcg@3 : 0.6757 , ndcg@5 : 0.7207 


100%|██████████| 15/15 [00:00<00:00, 16.69it/s]

epoch  1 test end : avg_loss = 0.0012
precision@1 : 0.8083 , precision@3 : 0.5715 , precision@5 : 0.3954 
ndcg@1 : 0.8083 , ndcg@3 : 0.7583 , ndcg@5 : 0.8017 
